# Interactive label distribution

### Common issues:
- `gv.Point` errors out with complaining about index
  - Probably one of the `vdims` is ill-defined
    - maybe an entry contains a list of values
- Caching of `gv.Point` class seems to perform quite bad
  - Usually just as fast to reproject directly than caching and loading result
    - spatialpandas: 26s without caching and spatialpandas
    - spatialpandas: 1min15 sek to cache
    - spatialandas: 2min to load from cache...
- If you are working with geographic data using GeoViews that needs to be projected before display and/or before datashading, GeoViews will have to do this every time you update a plot, which can drown out the performance improvement you get by using Datashader. GeoViews allows you to project the entire dataset at once using gv.operation.project, and once you do this you should be able to use Datashader at full speed.
- Not possible to do this via streamlit
  - https://discuss.streamlit.io/t/plotting-holoviews-plot/215

In [ ]:
import warnings

import cartopy.crs as ccrs
import datashader
import geopandas
import geoviews as gv
import holoviews as hv
import panel
import param
import spatialpandas
from holoviews.operation.datashader import datashade, dynspread
from shapely.errors import ShapelyDeprecationWarning
from natsort import natsorted

warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning)

gdf = geopandas.read_parquet("gdf/final_ben.parquet")
gdf.geometry = gdf.geometry.representative_point()
gdf = spatialpandas.GeoDataFrame(gdf)
gdf = gdf.explode("new_labels")
points = gv.Points(gdf, vdims=["new_labels"])
proj_points = gv.operation.project(points, projection=ccrs.GOOGLE_MERCATOR)
new_lbls = gdf["new_labels"].unique().tolist()
lbl_options = ["all"] + natsorted(new_lbls)
cmap_options = "blue red orange black gray".split()
tile_sources = {name: ts for name, ts in gv.tile_sources.tile_sources.items()}


class BenPanel(param.Parameterized):
    label = param.ObjectSelector(lbl_options[0], lbl_options)
    background = param.ObjectSelector(tile_sources["StamenTerrain"], tile_sources)
    cmap = "blue"
    # cmap_ = param.ObjectSelector(cmap_options[0], cmap_options)

    # @param.depends("cmap_")
    # def cmap(self):
    #     return self.cmap_

    @param.depends("label")
    def points(self):
        points = (
            proj_points
            if self.label == "all"
            else proj_points.select(new_labels=self.label)
        )
        return points

    @param.depends("background")
    def background_tile(self):
        return self.background

    def view(self, **kwargs):
        tile = hv.DynamicMap(self.background_tile)
        points = hv.DynamicMap(self.points)
        # cmap = hv.DynamicMap(self.cmap)
        # use aggregator `any`, as I am not interested in the density
        aggregator = datashader.any() if self.label == "all" else "default"
        shader = dynspread(
            datashade(points, cmap=self.cmap, aggregator=aggregator, precompute=True)
        )
        plot = tile * shader
        return plot.opts(
            width=800,
            height=800,
            toolbar="above",
            xaxis=None,
            yaxis=None,
            active_tools=["wheel_zoom"],
        )


ben = BenPanel(name="BigEarthNet Panel")
ben_panel = panel.Column(ben.param, ben.view()).servable()
#     ben_panel.save("interactive_ben.html", embed=True, max_states=20)
ben_panel